In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from gensim.models import KeyedVectors

In [ ]:
wv = KeyedVectors.load_word2vec_format('../input/fasttext-wikinews/wiki-news-300d-1M.vec')

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
train["words"] = train.excerpt.str.split('[^a-zA-Z0-9]')
test["words"] = test.excerpt.str.split('[^a-zA-Z0-9]')

In [ ]:
def words2wvs(words):
    wvs = []
    for word in words:
        if len(word) > 0:
            try:
                tmp = wv.get_vector(word)
                wvs.append(tmp)
            except:
                pass
    wvs = np.array(wvs)
    return wvs

In [ ]:
train["wv"] = train.words.map(words2wvs)
test["wv"] = test.words.map(words2wvs)

In [ ]:
def wv2centroid(wv):
    norm = np.linalg.norm(wv, axis=1)
    tmp = wv/norm[:, np.newaxis]
    return np.mean(tmp, axis=0)

In [ ]:
train["centroid"] = train.wv.map(wv2centroid)
test["centroid"] = test.wv.map(wv2centroid)

In [ ]:
X = np.vstack(train.centroid)
y = np.vstack(train.target).ravel()
test_X = np.vstack(test.centroid)
X.shape, y.shape, test_X.shape

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
svr = SVR()

random_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

svr_random = RandomizedSearchCV(estimator = svr, param_distributions = random_grid, cv = 3, verbose=2, random_state=1, n_jobs = -1)
# Fit the random search model
svr_random.fit(X, y)

In [ ]:
svr_random.best_params_, svr_random.best_score_

In [ ]:
y_pred = svr_random.predict(test_X)

In [ ]:
sub = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
sub

In [ ]:
sub.target = y_pred
sub.to_csv('submission.csv', index=False)